Selecting good features
http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everyting-side-by-side/

## stability selection

In [1]:
from sklearn.linear_model import RandomizedLasso
from sklearn.datasets import load_boston
boston = load_boston()

In [2]:
X, Y = boston['data'], boston['target']
names = boston['feature_names']

In [4]:
rlasso = RandomizedLasso(alpha=0.025)
rlasso.fit(X, Y)
print('features sorted by their scores:')
print(sorted(zip(map(lambda x:round(x, 4), rlasso.scores_),
                 names), reverse=True ))

features sorted by their scores:
[(1.0, 'RM'), (1.0, 'PTRATIO'), (1.0, 'LSTAT'), (0.65, 'B'), (0.57, 'CHAS'), (0.4, 'CRIM'), (0.375, 'TAX'), (0.22, 'NOX'), (0.215, 'DIS'), (0.135, 'INDUS'), (0.035, 'ZN'), (0.03, 'RAD'), (0.015, 'AGE')]


## recursive feature elimination

The stability of RFE depends heavily on the type of model that is used for feature ranking at each iteration.

In [7]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X, Y)
print('features sorted by their rank:')
print(sorted(zip(map(lambda x:round(x,4), rfe.ranking_), 
                 names)))

features sorted by their rank:
[(1.0, 'NOX'), (2.0, 'RM'), (3.0, 'CHAS'), (4.0, 'PTRATIO'), (5.0, 'DIS'), (6.0, 'LSTAT'), (7.0, 'RAD'), (8.0, 'CRIM'), (9.0, 'INDUS'), (10.0, 'ZN'), (11.0, 'TAX'), (12.0, 'B'), (13.0, 'AGE')]


## example: running the methods side by side

The dataset will be the so called Friedman #1 regression dataset (from Friedman’s Multivariate Adaptive Regression Splines paper). The data is generated according to formula y=10sin(πx1x2)+20(x3–0.5)2+10X4+5X5+ϵ, where the x1 to x5 are drawn from uniform distribution and ϵ is the standard normal deviate N(0,1). Additionally, the original dataset had five noise variables x6,…,x10, independent of the response variable. We will increase the number of variables further and add four variables x11,…,x14 each of which are very strongly correlated with x1,…,x4, respectively, generated by f(x)=x+N(0,0.01). This yields a correlation coefficient of more than 0.999 between the variables. This will illustrate how different feature ranking methods deal with correlations in the data.



In [10]:
from sklearn.datasets import load_boston
from sklearn.linear_model import (LinearRegression, Ridge, 
                                  Lasso, RandomizedLasso)
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#from minepy import MINE
np.random.seed(0)

In [23]:
size = 750
X = np.random.uniform(0, 1, (size, 14))

# friedamn no.1 regression problem
Y = (10 * np.sin(np.pi * X[:, 0] * X[:, 1]) +
     20 * (X[:, 2] - .5)**2 +
     10 * X[:, 3] + 5 * X[:, 4] + 
     np.random.normal(0, 1))
# add 3 additional correlated variables (correlated with X1-X3)
X[:, 10:] = X[:, :4] + np.random.normal(0, .025, (size, 4))

names = ['x%s' % i for i in range(1, 15)]

ranks = {}

def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order * np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks))

lr = LinearRegression(normalize=True)
lr.fit(X, Y)
ranks['LinReg'] = rank_to_dict(np.abs(lr.coef_), names)

ridge = Ridge(alpha=7)
ridge.fit(X, Y)
ranks['Ridge'] = rank_to_dict(np.abs(ridge.coef_), names)

lasso = Lasso(alpha=.05)
lasso.fit(X, Y)
ranks['Lasso'] = rank_to_dict(np.abs(lasso.coef_), names)

rlasso = RandomizedLasso(alpha=0.04)
rlasso.fit(X, Y)
ranks['Stab.'] = rank_to_dict(np.abs(rlasso.scores_), names)

# stop the search when 5 features are left
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(X, Y)
ranks['RFE'] = rank_to_dict(map(float, rfe.ranking_), names, order=-1)

rf = RandomForestRegressor()
rf.fit(X, Y)
ranks['RF'] = rank_to_dict(rf.feature_importances_, names)

f, pval = f_regression(X, Y, center=True)
ranks['Corr.'] = rank_to_dict(f, names)

# mine = MINE()
# mic_scores = []
# for i in range(X.shape[1]):
#     mine.compute_score(X[:, i], Y)
#     m = mine.mic()
#     mic_scores.append(m)
# ranks['MIC'] = rank_to_dict(mic_scores, names)

r = {}
for name in names:
    r[name] = round(np.mean([ranks[method][name] for method in ranks.keys()]), 2)

methods = sorted(ranks.keys())
ranks['Mean'] = r
methods.append('Mean')

print('\t%s' % '\t'.join(methods))
for name in names:
    print('%s\t%s' % (name, '\t'.join(map(str, [ranks[method][name] for method in methods]))))

	Corr.	Lasso	LinReg	RF	RFE	Ridge	Stab.	Mean
x1	0.32	0.73	0.42	0.16	1.0	0.65	0.68	0.57
x2	0.27	0.94	1.0	0.4	1.0	0.7	0.65	0.71
x3	0.0	0.0	0.19	0.09	0.78	0.01	0.0	0.15
x4	1.0	1.0	0.56	0.28	1.0	1.0	1.0	0.83
x5	0.12	0.65	0.39	0.19	1.0	0.87	0.66	0.55
x6	0.0	0.0	0.01	0.01	0.11	0.0	0.0	0.02
x7	0.0	0.0	0.01	0.01	0.22	0.03	0.0	0.04
x8	0.01	0.0	0.0	0.0	0.0	0.02	0.0	0.0
x9	0.0	0.0	0.02	0.01	0.33	0.05	0.0	0.06
x10	0.0	0.0	0.03	0.01	0.44	0.08	0.0	0.08
x11	0.31	0.22	0.11	0.46	0.56	0.63	0.57	0.41
x12	0.26	0.0	0.46	0.2	1.0	0.58	0.34	0.41
x13	0.0	0.0	0.18	0.03	0.67	0.01	0.0	0.13
x14	0.99	0.51	0.27	1.0	0.89	0.98	0.5	0.73


In [20]:
ranks['Mean']['x1']

0.41